### Abstract

This notebook demonstrates how to convert the models from sklearn to ONNX and do the inference from ONNX runtime.


#### References:

- [sklearn-onnx](http://onnx.ai/sklearn-onnx/)

- [github sklearn-onnx](https://github.com/onnx/sklearn-onnx)

### Train Model

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Convert Model to ONNX

In [11]:
# Convert into ONNX format with onnxmltools
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([1, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

The maximum opset needed by this model is only 9.
The maximum opset needed by this model is only 1.


In [6]:
!ls -hl

total 12K
-rw-r--r-- 1 lguo ama-unix 7.0K Jul 24 14:15 rf_iris.onnx
-rw-r--r-- 1 lguo ama-unix 2.9K Jul 24 14:15 sklearn-onnx.ipynb


### Inference with ONNX

In [21]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

ImportError: cannot import name 'RunOptions'